In [ ]:
datadir = ""
scan_numbers = ""
step = [0.002]

# Run MSMapper and Analyse Results

In [1]:
import sys, os
import shutil
import h5py
import numpy as np
import matplotlib.pyplot as plt

from i16_msmapper import mapper_runner, module_info

print("sys.path: ", sys.path)
print(module_info())

# set the location of msmapper
CONFIG = '/tmp/msmapper_config'
MSMAPPER = '/opt/msmapper/msmapper'
OUTPUT = '/tmp/msmapper_result.png'
OUTPUT_NX = '/tmp/msmapper_result.nxs'
mapper_runner.SHELL_CMD = ' '.join([MSMAPPER, "-configuration", CONFIG, "-bean %s"])

if not os.path.exists(MSMAPPER):
    raise ValueError("MSMAPPER not found")

# make config dir
os.makedirs(CONFIG, exist_ok=True)


Writable TEMPDIR = /tmp
sys.path:  ['/scratch/grp66007/python/magnetic-materials-workflows', '/scratch/grp66007/software/pycharm/pycharm-2025.2/plugins/python-ce/helpers/pydev', '/scratch/grp66007/software/pycharm/pycharm-2025.2/plugins/python-ce/helpers/jupyter_debug', '/dls/science/users/grp66007/envs/mmg_toolbox/lib/python312.zip', '/dls/science/users/grp66007/envs/mmg_toolbox/lib/python3.12', '/dls/science/users/grp66007/envs/mmg_toolbox/lib/python3.12/lib-dynload', '', '/home/grp66007/.local/lib/python3.12/site-packages', '/dls/science/users/grp66007/envs/mmg_toolbox/lib/python3.12/site-packages']
path:  
Python version 3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:24:40) [GCC 13.3.0]
i16_msmapper version 1.6.2 (14/08/25)
  msmapper version: Not available
     numpy version: 2.2.2
matplotlib version: 3.10.0
hdfmap version: 1.0.0 (2025/07/23)
   tkinter version: 8.6



ValueError: MSMAPPER not found

In [ ]:
# Inputs
processing_directory = os.path.join(datadir, "processing")
if not os.path.isdir(datadir):
    raise ValueError("data directory not found")
if not os.path.isdir(processing_directory):
    raise ValueError("processing directory not found")

scan_numbers = np.fromstring(scan_numbers.strip('[]'), dtype=int, sep=',')
inpath = [os.path.join(datadir, f"{n}.nxs") for n in scan_numbers]
for file in inpath:
    if not os.path.exists(file):
        raise ValueError(f"file not found: {file}")

if len(scan_numbers) == 1:
    outpath = os.path.join(processing_directory, f"{scan_numbers[0]}_workflows_msmapper.nxs")
else:
    outpath = os.path.join(processing_directory, f"{scan_numbers[0]}-{scan_numbers[-1]}_workflows_msmapper.nxs")
print("inpath: ", inpath)
print("outpath: ", outpath)

step = np.fromstring(step.strip('[]'), sep=',').tolist()
print(f"step = {step}")

## Run MSMapper

In [ ]:
mapper_runner.run_msmapper(
    input_files=inpath,
    output_file=outpath,
    step=step,
    reduce_box=True,
    normalisation='rc',
)
print(f"File created {outpath}: {os.path.isfile(outpath)}")
print(f"File size: {os.stat(outpath).st_size/1e6} MB")
if os.path.isfile(outpath):
    shutil.copy(outpath, OUTPUT_NX)


## Load MSMapper output
View the NeXus file [here](https://myhdf5.hdfgroup.org/) (drag and drop your file).

In [ ]:
filename = os.path.basename(outpath)

with h5py.File(outpath, 'r') as hdf:
    # the following are links to the original scan file
    scan_command = hdf['/entry0/scan_command'].asstr()[()]  # str
    crystal = hdf['/entry0/sample/name'].asstr()[()]  # str
    temp = hdf.get('/entry0/instrument/temperature_controller/Tsample', np.array(0))[()]  # float
    unit_cell = np.reshape(hdf['/entry0/sample/unit_cell'], -1)  # 1D array, length 6
    energy = hdf['/entry0/sample/beam/incident_energy'][()]  # # float
    ubmatrix = hdf['/entry0/sample/ub_matrix'][0]  # 3D array, shape (3,3)
    # this is the processed data
    haxis = hdf['/processed/reciprocal_space/h-axis'][()]  # 1D array, length n
    kaxis = hdf['/processed/reciprocal_space/k-axis'][()]  # 1D array, length m
    laxis = hdf['/processed/reciprocal_space/l-axis'][()]  # 1D array, length o
    volume = hdf['/processed/reciprocal_space/volume'][()]  # 3D array, shape [n,m,o]
    # detector parameters
    pixel_size = hdf['/entry0/instrument/pil3_100k/module/fast_pixel_direction'][()] # float, mm
    detector_distance = hdf['/entry0/instrument/pil3_100k/transformations/origin_offset'][()] # float, mm

print(f"Loaded file: {filename} with volume shape: {volume.shape}")

print(f"scan_command: {scan_command}")
print(f"crystal: {crystal}")
print(f"temp: {temp}")
print(f"unit_cell: {unit_cell}")
print(f"energy: {energy}")
print(f"ubmatrix: {ubmatrix}")
print(f"haxis: {haxis.min()}:{haxis.max()} {haxis.shape}")
print(f"kaxis: {kaxis.min()}:{kaxis.max()} {kaxis.shape}")
print(f"laxis: {laxis.min()}:{laxis.max()} {laxis.shape}")
print(f"volume: {volume.shape}")
print(f"pixel_size: {pixel_size}")
print(f"detector_distance: {detector_distance}")


In [ ]:
# average angle subtended by each pixel
solid_angle = pixel_size ** 2 / detector_distance ** 2  # sr
if solid_angle.size > 0:
    solid_angle = solid_angle[0]
print(f'Each pixel is normalised by the solid angle: {solid_angle: .4g} sr')

volume = volume * solid_angle

In [ ]:
# Plot summed cuts
plt.figure(figsize=(18, 8), dpi=60)
title = f"{filename} '{crystal}' {temp:.3g} K\n{scan_command}"
plt.suptitle(title, fontsize=18)

plt.subplot(131)
plt.plot(haxis, volume.sum(axis=1).sum(axis=1))
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [1,2]', fontsize=16)

plt.subplot(132)
plt.plot(kaxis, volume.sum(axis=0).sum(axis=1))
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,2]', fontsize=16)

plt.subplot(133)
plt.plot(laxis, volume.sum(axis=0).sum(axis=0))
plt.xlabel('l-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,1]', fontsize=16)

# Plot summed images
plt.figure(figsize=(18, 8), dpi=60)
title = f"{filename}\n{crystal} {temp:.3g} K: {scan_command}"
plt.suptitle(title, fontsize=20)
plt.subplots_adjust(wspace=0.3)

plt.subplot(131)
K, H = np.meshgrid(kaxis, haxis)
plt.pcolormesh(H, K, volume.sum(axis=2), shading='auto')
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('k-axis (r.l.u.)', fontsize=16)
plt.axis('image')
#plt.colorbar()

plt.subplot(132)
L, H = np.meshgrid(laxis, haxis)
plt.pcolormesh(H, L, volume.sum(axis=1), shading='auto')
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
plt.axis('image')
#plt.colorbar()

plt.subplot(133)
L, K = np.meshgrid(laxis, kaxis)
plt.pcolormesh(K, L, volume.sum(axis=0), shading='auto')
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
plt.axis('image')
plt.colorbar()
plt.savefig(OUTPUT)
plt.show()

## NXtransformations
plot reciprocal space and instrument positions from NXtransformations

In [ ]:

from i16_msmapper.nx_transformations import NXScan

with h5py.File(inpath) as nxs:
    scan = NXScan(nxs)
    scan.plot_wavevectors()

    scan.plot_instrument()

    plt.show()